In [ ]:
# ===== CONSTANTS =====
# Hier werden Konstanten definiert, die im gesamten Programm verwendet werden.
# cache_file: Name der Datei, in die gefilterte/sortierte Daten geschrieben werden.
# source_file: Name der Ursprungsdatei mit allen Daten.
cache_file = "cache.csv"
source_file = "names.csv"

# ===== CORE FUNCTIONS =====
# Diese Funktionen bilden das Herzstück der Datenverarbeitung.

# Gibt Header (Spaltennamen) und alle Zeilen einer CSV-Datei zurück.
def get_content(file):
    # Öffne die Datei im Lesemodus. Gibt Exception, wenn Datei nicht existiert.
    with open(file, "r") as data:
        lines = data.readlines()  # Lese alle Zeilen in eine Liste
        header = lines[0].strip().split(",")  # Erste Zeile = Header
    return header, lines

# Schreibt Header und Zeilen in die Cache-Datei.
def generate_cache_file(header, lines, file = cache_file):
    # Öffne die Datei im Schreibmodus, überschreibt vorhandene Datei.
    with open(file, "w") as new_file:
        new_file.write(",".join(header) + "\n")  # Schreibe Header
        for line in lines:
            # Jede Zeile bereits als Liste -> füge mit Kommata zusammen
            new_file.write(",".join(line) + "\n")

# Druckt die Datei als formatierte Tabelle in der Konsole.
def print_formatted_table(file = cache_file):
    header, lines = get_content(file)
    # Überschrift und Anzahl der Einträge anzeigen
    print("\nFormatted Table, from file:", file)
    print("With a total of " + str(len(lines)-1) +" entrys.")
    # Trennlinie basierend auf Anzahl der Spalten
    print("-" * round(23.2 * len(header)))
    # Header-Zeile, Spaltenbreiten festgelegt auf 20 Zeichen
    print(("| {:<20} " * len(header) + "|").format(*header))
    print("-" * round(23.2 * len(header)))
    # Datenzeilen: Splitte jede Zeile an Kommas und formatiere
    for line in lines[1:]:
        print(("| {:<20} " * len(header) + "|").format(*line.strip().split(",")))

# Gibt nur Zeilen zurück, bei denen eine Spalte einen der erlaubten Werte enthält.
def get_rows_by_value(column, value, file = cache_file, return_table = False,):
    header, lines = get_content(file)
    # Prüfe, ob die Spalte existiert
    if column not in header:
        print("Spalte '" + column + "' nicht gefunden.")
        return
    col_index = header.index(column)  # Position der Spalte
    rows = []
    # Durchlaufe alle Datenzeilen
    for line in lines[1:]:
        row = line.strip().split(",")
        if row[col_index] in value:
            rows.append(row)  # Füge passende Zeile hinzu
    # Schreibe gefilterte Zeilen in cache_file
    generate_cache_file(header, rows)
    # Gib Filter-Ergebnis in Konsole aus
    print("\nOnly returned rows with value in column:", column)
    print("For Values:", value)
    if return_table:
        print_formatted_table()

# Sortiert Zeilen nach der angegebenen Spalte mit Bubble Sort (Lerneffekt).
# reverse=True für absteigende Reihenfolge.
def sort_by(column, area, reverse=False, file=cache_file, return_table=False):
    header, lines = get_content(file)
    if column not in header:
        print("Spalte '" + column +"' nicht gefunden.")
        return
    if area == []:
        area = [1, len(lines)]  # Gesamter Bereich
    # Wandle Zeilen in Listen um, auf die geschnitten wird
    rows = [line.strip().split(",") for line in lines[area[0] : area[1]]]
    col_index = header.index(column)
    # Bubble Sort: Vergleiche jeweils benachbarte Einträge
    for i in range(len(rows)):
        for j in range(0, len(rows) - i -1):
            a = int(rows[j][col_index])
            b = int(rows[j + 1][col_index])
            # Tausch, wenn falsche Reihenfolge
            if (a > b and not reverse) or (a < b and reverse):
                rows[j], rows[j + 1] = rows[j + 1], rows[j]
    # Schreibe sortierte Daten zurück in cache_file
    generate_cache_file(header, rows)
    print("\nSorted by column:", column)
    print("Range:", area[0], "to", area[1])
    if return_table:
        print_formatted_table()

# Summiert alle Werte in der Spalte 'Number' (Häufigkeiten).
def count_humans(file = cache_file):
    header, lines = get_content(file)
    col_index = header.index("Number")
    total_humans = 0
    for line in lines[1:]:
        row = line.strip().split(",")
        total_humans += int(row[col_index])
    return total_humans

# Berechnet Durchschnittsalter: aktuelles Jahr minus Geburtsjahr.
def average_age(file=cache_file):
    header, lines = get_content(file)
    col_index = header.index("YearOfBirth")
    current_year = 2025  # Fest kodiert, könnte mit datetime.now().year dynamisch sein
    total_age = 0
    count = 0
    for line in lines[1:]:
        row = line.strip().split(",")
        age = current_year - int(row[col_index])
        total_age += age
        count += 1
    # Vermeide Division durch Null
    if count == 0:
        print("No data available to calculate average age.")
        return None
    avg_age = total_age / count
    return avg_age

# ===== MAIN FUNCTION =====
def main():
    # Starte Endlosschleife für Menüsteuerung
    while True:
        # Abfrage: Filterung durchführen?
        filter_choice = input("\nWould you like to filter the data? (yes/no/quit): ").lower()
        if filter_choice == 'quit':
            print("Exiting program...")
            break
        if filter_choice == 'yes':
            # Spaltennamen und Werte abfragen
            print("\nAvailable columns to filter by: StateCode, YearOfBirth, Sex, Number")
            column = input("Enter column name to filter by: ")
            values = input("Enter values to filter for (comma-separated): ").split(',')
            values = [v.strip() for v in values]
            # Filter anwenden
            get_rows_by_value(column, values, source_file)
            print("\nFilter applied successfully!")
            # Gefilterte Daten anzeigen?
            if input("Would you like to see the filtered data? (yes/no): ").lower() == 'yes':
                print_formatted_table(cache_file)
            # Statistiken anzeigen?
            if input("\nWould you like to see statistics? (yes/no): ").lower() == 'yes':
                print(f"\nTotal humans in filtered data: {count_humans(cache_file)}")
                avg = average_age(cache_file)
                if avg is not None:
                    print(f"Average age: {avg:.1f} years")
            # Sortieren anbieten?
            if input("\nWould you like to sort the data? (yes/no): ").lower() == 'yes':
                sort_column = input("Enter column to sort by: ")
                start = int(input("Enter start row (default 1): ") or 1)
                end = int(input("Enter end row (default all): ") or -1)
                direction = input("Sort direction (asc/desc): ").lower()
                sort_by(sort_column, [start, end], direction == 'desc', cache_file, True)
        elif filter_choice == 'no':
            # Originaldaten-Statistiken anzeigen
            print("\nShowing statistics for original data:")
            print(f"Total humans: {count_humans(source_file)}")
            avg = average_age(source_file)
            if avg is not None:
                print(f"Average age: {avg:.1f} years")
            continue
        else:
            print("Please enter 'yes', 'no', or 'quit'")
            continue

# ===== ENTRY POINT =====
if __name__ == "__main__":
    main()
